In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
import torch.nn as nn
import torch.optim as optim
from random import shuffle
import wandb
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import SubsetRandomSampler
import torchvision

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-08 05:23:27--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.207, 173.194.210.207, 173.194.211.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G   200MB/s    in 18s     

2024-04-08 05:23:46 (198 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import SubsetRandomSampler


class CNN(nn.Module):
    def __init__(self, img_size, n_filters, filter_size, fc_size, drop_out, n_classes, activation):
        super(CNN, self).__init__()
        self.img_size = img_size
        self.activation = self._get_activation_function(activation)

        self.conv1 = nn.Conv2d(3, n_filters[0], kernel_size=filter_size[0], stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(n_filters[0])
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(n_filters[0], n_filters[1], kernel_size=filter_size[1], stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(n_filters[1])

        self.conv3 = nn.Conv2d(n_filters[1], n_filters[2], kernel_size=filter_size[2], stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(n_filters[2])

        self.conv4 = nn.Conv2d(n_filters[2], n_filters[3], kernel_size=filter_size[3], stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(n_filters[3])

        self.conv5 = nn.Conv2d(n_filters[3], n_filters[4], kernel_size=filter_size[4], stride=1, padding=0)
        self.bn5 = nn.BatchNorm2d(n_filters[4])

        self.drop_out = nn.Dropout(drop_out)

        conv_out_size = self._get_conv_out()
        self.fc1 = nn.Linear(conv_out_size, fc_size)
        self.fc2 = nn.Linear(fc_size, n_classes)

    def forward(self, x):
        x = self.pool(self.bn1(self.activation(self.conv1(x))))
        x = self.pool(self.bn2(self.activation(self.conv2(x))))
        x = self.pool(self.bn3(self.activation(self.conv3(x))))
        x = self.pool(self.bn4(self.activation(self.conv4(x))))
        x = self.pool(self.bn5(self.activation(self.conv5(x))))
        x = self.drop_out(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.activation(self.fc1(x))
        x = nn.Softmax(dim=1)(self.fc2(x))
        return x

    def _get_conv_out(self):
        x = torch.rand(1, 3, self.img_size, self.img_size)
        x = self.pool(self.bn1(self.activation(self.conv1(x))))
        x = self.pool(self.bn2(self.activation(self.conv2(x))))
        x = self.pool(self.bn3(self.activation(self.conv3(x))))
        x = self.pool(self.bn4(self.activation(self.conv4(x))))
        x = self.pool(self.bn5(self.activation(self.conv5(x))))
        return x.view(1, -1).size(1)

    def _get_activation_function(self, activation):
        if activation == 'relu':
            return nn.ReLU()
        elif activation == 'elu':
            return nn.ELU()
        elif activation == 'silu':
            return nn.SiLU()
        elif activation == 'selu':
            return nn.SELU()
        elif activation == 'gelu':
            return nn.GELU()


def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    return epoch_loss, epoch_accuracy


def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    loss = running_loss / len(data_loader.dataset)
    accuracy = correct_predictions / total_predictions
    return loss, accuracy


def fit_model(img_size, n_filters, filter_size, fc_size, drop_out, n_classes, learning_rate, num_epochs,
              activation, optimizer_type):

    criterion = nn.CrossEntropyLoss()
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_data_path = "/kaggle/working/inaturalist_12K/train"
    test_data_path = "/kaggle/working/inaturalist_12K/val"
    train_dataset = datasets.ImageFolder(train_data_path, transform=transform)
    val_size = int(0.1 * len(train_dataset))
    class_indices = {class_idx: [] for class_idx in range(len(train_dataset.classes))}
    for idx, (_, label) in enumerate(train_dataset):
        class_indices[label].append(idx)
    val_indices = []
    for class_idx, indices in class_indices.items():
        val_indices.extend(indices[:val_size // len(train_dataset.classes)])
    val_sampler = SubsetRandomSampler(val_indices)
    batch_size = 512
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)

    model = CNN(img_size, n_filters, filter_size, fc_size, drop_out, n_classes, activation).to(device)

    if optimizer_type == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_type == 'nadam':
        optimizer = optim.NAdam(model.parameters(), lr=learning_rate)
    elif optimizer_type == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        train_loss, accuracy = train(model, train_loader, criterion, optimizer, device)
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {accuracy:.4f}")
        wandb.log({"train_loss":train_loss,"train_acc":accuracy,"Epoch":epoch})

        val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        wandb.log({"Val_loss":val_loss,"Val_acc":val_accuracy})

    test_dataset = datasets.ImageFolder(test_data_path, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    return model


# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img_size = 224
n_filters = [16, 32, 64, 128, 256]
filter_size = [3, 3, 3, 3, 3]
fc_size = 512
drop_out = 0.5
n_classes = 10
learning_rate = 0.0001
num_epochs = 10
activation = 'relu'
optimizer_type = 'adam'

# Fit the model
trained_model = fit_model(img_size, n_filters, filter_size, fc_size, drop_out, n_classes, learning_rate, num_epochs,
                     activation, optimizer_type)


Epoch 1/10, Train Loss: 2.2214, Train Accuracy: 0.2256
Validation Loss: 0.2274, Validation Accuracy: 0.1141


In [ ]:
import wandb
# socket.setdefaulttimeout(30)
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ··································


wandb: ERROR API key must be 40 characters long, yours was 33
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def main():

    wandb.init(project='CS6910_ass2')
    config = wandb.config
    trained_model = fit_model(config.img_size, config.n_filters, config.filter_size,
                              config.fc_size, config.drop_out, config.n_classes,
                              config.learning_rate, config.num_epochs,
                           config.activation, config.optimizer_type)
    wandb.finish()

sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "img_size": {"values": [128, 224, 256]},
        "n_filters": {"values": [[16, 32, 64, 128, 256], [32, 64, 128, 256, 512]]},
        "filter_size": {"values": [[3, 3, 3, 3, 3], [5, 5, 5, 5, 5]]},
        "fc_size": {"values": [256, 512, 1024]},
        "drop_out": {"values": [0.2, 0.3, 0.5]},
        "n_classes": {"values": [5, 10, 20]},
        "learning_rate": {"values": [0.0001, 0.001, 0.01]},
        "num_epochs": {"values": [5, 10, 15]},
        "activation": {"values": ["relu", "elu", "silu", "selu", "gelu"]},
        "optimizer_type": {"values": ["adam", "nadam", "rmsprop"]},
        "batch_size": {"values": [64, 128, 256]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="CS6910_ass2")

# Run sweep
wandb.agent(sweep_id, function=main)